In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [3]:
from torchvision import datasets, transforms
data_path = '/data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(size=[32,32], padding=4),
        transforms.RandomHorizontalFlip(),                       
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))


Extracting /data-unversioned/p1ch6/cifar-10-python.tar.gz to /data-unversioned/p1ch6/


In [4]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [5]:
class resblock(nn.Module):
    def __init__(self, n):
        super(resblock, self).__init__()
        self.conv = nn.Conv2d(n, n, kernel_size=3,
                              padding=1, bias=False)  
        self.batch_norm = nn.BatchNorm2d(num_features=n)

        torch.nn.init.kaiming_normal_(self.conv.weight,
                                      nonlinearity='relu')  
        torch.nn.init.constant_(self.batch_norm.weight, 0.5)
        torch.nn.init.zeros_(self.batch_norm.bias)

    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = torch.relu(out)
        return out + x

In [6]:
class deep(nn.Module):
    def __init__(self, n1=32, n_blocks=18):
        super().__init__()
        self.n1 = n1
        self.conv1 = nn.Conv2d(3, n1, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(n1, 2*n1, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(2*n1, 4*n1, kernel_size=3, padding=1)
        self.resblocks = nn.Sequential(
            *(n_blocks * [resblock(n=n1)]))
        self.resblocks2 = nn.Sequential(
            *(n_blocks * [resblock(n=n1*2)]))
        self.resblocks3 = nn.Sequential(
            *(n_blocks * [resblock(n=n1*4)]))
        self.fc1 = nn.Linear(8 * 8 * n1*4, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 10)
        
    def forward(self, x):
        out = torch.relu(self.conv1(x))
        out = self.resblocks(out)
        out = F.max_pool2d(out, 2) 
        out = self.conv2(out)
        out = self.resblocks2(out)
        out = F.max_pool2d(out, 2) 
        out = self.conv3(out)
        out = self.resblocks3(out)
        out = out.view(-1, 8 * 8 * self.n1*4)
        out = torch.relu(self.fc1(out))
        out = torch.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [7]:
device = (torch.device('cuda') if torch.cuda.is_available()
          else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cuda.


In [8]:
import datetime

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader, schedular):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            imgs = imgs.to(device=device)  # <1>
            labels = labels.to(device=device)
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_train += loss.item()

        schedular.step() 
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))

In [9]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=True)

model =  deep().to(device=device)# <1>
optimizer = optim.SGD(model.parameters(), lr=1e-2, weight_decay=0.01)
schedular = optim.lr_scheduler.MultiStepLR(optimizer, [50,85], gamma=0.1)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
from torchsummary import summary
summary(model,(3,32,32))

In [11]:
training_loop(  
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader, 
    schedular = schedular
)

2021-07-31 07:26:45.053442 Epoch 1, Training loss 1.8153790540402504
2021-07-31 07:28:10.158411 Epoch 2, Training loss 1.341204586617477
2021-07-31 07:29:35.879146 Epoch 3, Training loss 1.1142738347163286
2021-07-31 07:31:02.142383 Epoch 4, Training loss 0.9822061436865336
2021-07-31 07:32:28.084055 Epoch 5, Training loss 0.8956750226600091
2021-07-31 07:33:54.329051 Epoch 6, Training loss 0.837829377454565
2021-07-31 07:35:20.170078 Epoch 7, Training loss 0.7866853319318093
2021-07-31 07:36:45.995773 Epoch 8, Training loss 0.7376360129898466
2021-07-31 07:38:12.074362 Epoch 9, Training loss 0.7048707171855375
2021-07-31 07:39:38.099320 Epoch 10, Training loss 0.6660417033659528
2021-07-31 07:41:03.965688 Epoch 11, Training loss 0.6453866245572829
2021-07-31 07:42:29.227396 Epoch 12, Training loss 0.6115765212213292
2021-07-31 07:43:54.489330 Epoch 13, Training loss 0.595123569735938
2021-07-31 07:45:20.259157 Epoch 14, Training loss 0.5708884028598781
2021-07-31 07:46:45.520342 Epoch

In [12]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.5f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.97298
Accuracy val: 0.89940


In [13]:
optimizer = optim.SGD(model.parameters(), lr=0.0001, weight_decay=0.01)

In [14]:
training_loop(  
    n_epochs = 10,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader, 
    schedular = schedular
)

2021-07-31 09:55:33.904073 Epoch 1, Training loss 0.08734519359753336
2021-07-31 09:56:59.719448 Epoch 2, Training loss 0.08545545311620854
2021-07-31 09:58:25.321029 Epoch 3, Training loss 0.08339083113509904
2021-07-31 09:59:50.912315 Epoch 4, Training loss 0.08179607545204289
2021-07-31 10:01:16.637043 Epoch 5, Training loss 0.08269403505322459
2021-07-31 10:02:42.080491 Epoch 6, Training loss 0.0811040647115911
2021-07-31 10:04:07.845876 Epoch 7, Training loss 0.08199799516121559
2021-07-31 10:05:33.477687 Epoch 8, Training loss 0.08176422959474651
2021-07-31 10:06:59.865838 Epoch 9, Training loss 0.07950862104077931
2021-07-31 10:08:25.596955 Epoch 10, Training loss 0.07886941943203321


In [15]:
validate(model, train_loader, val_loader)

Accuracy train: 0.97442
Accuracy val: 0.90040
